   生成输入数据集 

In [ ]:
from property_generator import*

N_SAMPLE= 16
N_WELL=1
N_TASKS = 32
NUM_CONTRAL_POINT=10
PORO_VAR=0.004
PERM_VAR=8100
LEN_SCALE=[3200, 3200, 80]
x = np.arange(1900, 8200, 200)#32 6400
y = np.arange(1100, 7400, 200)#32 6400
z = np.arange(-2975, -2820, 10)#16 160


STEPS=[0,1,3,9,16,22,28,34,40,50]
T_STOP=28
INJ_RATE=15

GetAllTask(x,y,z,N_TASKS, N_SAMPLE,STEPS,T_STOP, INJ_RATE,PORO_VAR, PERM_VAR, LEN_SCALE, NUM_CONTRAL_POINT)

In [ ]:
import subprocess

path_comsol_server="D:\comsol\COMSOL62\Multiphysics\bin\win64\comsolmphserver.exe"####检查修改COMSOL的路径设置，符合你自己的PC设置
comsol_server_process = subprocess.Popen(path_comsol_server)
cmd_matlb_script="matlab -nosplash -nodesktop -r StarCOMSOL" ####检查修改StarCOMSOL中的路径设置，符合你自己的PC设置
os.system(cmd_matlb_script)
comsol_server_process.wait()

In [ ]:
from cat_label import CatLabels
cols=np.linspace(3, 21, 10, dtype=np.int16)
catlabels = CatLabels("./", "new312","./dataSet",x,y,z,cols)
catlabels.DealAllTask()

进行训练

In [ ]:
from normal_train import*
from para import*

def plot(log, selected_epoch, name='log'):
    fig,ax = plt.subplots(figsize=(6, 4))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('loss')
    if len(np.shape(log))==1:
        x=np.arange(1,len(log)+1)
        y=log
        ax.plot(x, y)
    else:
        x=np.arange(1,len(log[0])+1)
        for row in selected_epoch:
            y=log[row]
            ax.plot(x, y, label=f'epoch{row}')
    fig.legend()
    fig.savefig(f'{name}.png',dpi=500)
    fig.show()

def plot_compare(log1, log2, name='log'):
    fig,ax = plt.subplots(figsize=(6, 4))
    ax.set_xlabel('Epoch')
    ax.set_ylabel(f'{name}') 
    x=np.arange(1,len(log1)+1)
    y=log1
    ax.plot(x, y, label=f'reptile')
    y=log2
    ax.plot(x, y, label=f'reptile')
    fig.savefig(f'{name}.png',dpi=500)
    fig.show()

In [ ]:


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

ns_ls=[16,32,64]
dataset=NomalCo2Dataset_loadALL("./target_task/data_normal")
lr=0.001
for ns in ns_ls:
    print(f"-----------------------------------------normalloss_mse_lr{lr}_ns{ns}---------------------------------------------")
    ns_=63-ns
    model_path_normal = f"./saved_model/normal_fineds_mse_lr{lr}_ns{ns}.pth"
    dataset_i, _= torch.utils.data.random_split(dataset, [ns, ns_])
    model = build_resunetplusplus()
    model = model.to(DEVICE)
    loss_normal=normal_train(model, dataset_i, 0.25, EPOCH, BATCH_SIZE, lr, DEVICE,model_path_normal)
    np.savetxt(f"normalloss_mse_lr{lr}_ns{ns}.csv",loss_normal,delimiter=',')
    plot(loss_normal, [0], f'normalloss_mse_lr{lr}_ns{ns}.png')
    

cuda:0
Avaliable samples: 63
-----------------------------------------normalloss_mse_lr0.001_ns16---------------------------------------------
epoch0  eval loss:0.005092544015496969 r2:-2.411942720413208
epoch1  eval loss:0.012686465866863728 r2:-7.499778747558594
epoch2  eval loss:0.012549427337944508 r2:-7.407964706420898
epoch3  eval loss:0.008149559609591961 r2:-4.460106372833252
epoch4  eval loss:0.0033270777203142643 r2:-1.2291016578674316
epoch5  eval loss:0.0026831079740077257 r2:-0.7976498603820801
epoch6  eval loss:0.002145608654245734 r2:-0.43753182888031006
epoch7  eval loss:0.0023064687848091125 r2:-0.5453062057495117
epoch8  eval loss:0.0018715382320806384 r2:-0.2539079189300537
epoch9  eval loss:0.0016514426097273827 r2:-0.10644650459289551
epoch10  eval loss:0.0018856695387512445 r2:-0.26337575912475586
epoch11  eval loss:0.0018744695698842406 r2:-0.25587189197540283
epoch12  eval loss:0.001598075614310801 r2:-0.07069134712219238
epoch13  eval loss:0.001505133113823831 

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


-----------------------------------------normalloss_mse_lr0.001_ns32---------------------------------------------
epoch0  eval loss:0.030209019780158997 r2:-27.811105728149414
epoch1  eval loss:0.01786147430539131 r2:-10.95819091796875
epoch2  eval loss:0.004468063358217478 r2:-3.191244602203369
epoch3  eval loss:0.0034866956993937492 r2:-2.1236419677734375
epoch4  eval loss:0.0014850350562483072 r2:-0.24340999126434326
epoch5  eval loss:0.0018217936158180237 r2:-1.5581269264221191
epoch6  eval loss:0.0017175044631585479 r2:-0.173805832862854
epoch7  eval loss:0.0015665924875065684 r2:-0.25426673889160156
epoch8  eval loss:0.0014527362072840333 r2:-0.1719902753829956
epoch9  eval loss:0.0016048422548919916 r2:-0.047772884368896484
epoch10  eval loss:0.00137109006755054 r2:-0.13215851783752441
epoch11  eval loss:0.0011896002106368542 r2:-0.08572936058044434
epoch12  eval loss:0.0013361419551074505 r2:-0.07571887969970703
epoch13  eval loss:0.0012585707008838654 r2:-0.057744741439819336


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


-----------------------------------------normalloss_mse_lr0.001_ns64---------------------------------------------
epoch0  eval loss:0.015672652050852776 r2:-12.048540115356445
epoch1  eval loss:0.004159256815910339 r2:-1.542574167251587
epoch2  eval loss:0.0022625781130045652 r2:-0.37586867809295654
epoch3  eval loss:0.0017043785192072392 r2:-0.3733339309692383
epoch4  eval loss:0.0017785015515983105 r2:-0.11846065521240234
epoch5  eval loss:0.001671105157583952 r2:-0.08650338649749756
epoch6  eval loss:0.0017132129287347198 r2:-0.07522034645080566
epoch7  eval loss:0.0017064193962141871 r2:-0.03360772132873535
epoch8  eval loss:0.0016428617527708411 r2:-0.03759598731994629
epoch9  eval loss:0.001510458067059517 r2:-0.04869246482849121
epoch10  eval loss:0.001656778622418642 r2:-0.00886523723602295
epoch11  eval loss:0.0015773200429975986 r2:-0.02069556713104248
epoch12  eval loss:0.0015993925044313073 r2:0.006013035774230957
epoch13  eval loss:0.0015395913505926728 r2:0.04476219415664

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [ ]:
normal_loss=np.loadtxt(f"normalloss_wz5_sc100_lr0.0005_ns16.csv",loss_normal,delimiter=',')
plot(loss_normal, [0], 'normal_loss')

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [ ]:
from reptile_train import *
from para import*
from normal_train import*

####
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)
####
LR_OUTER=1e-06
EPOCH_INNER=3

model_path_reptile =f"./saved_model/new_reptile_MSE_lro{LR_OUTER}_ei{EPOCH_INNER}.pth"
testset=MateCo2Dataset("./data_meta", 12)
model = build_resunetplusplus()
model = model.to(DEVICE)
train_loader = DataLoader(testset, batch_size=1, shuffle=True)
targetset=MateCo2Dataset("./target_task/data_meta", 48)
eval_loader = DataLoader(targetset, batch_size=1, shuffle=False)
loss_reptile=outter_loop(model, train_loader, EPOCH_OUTTER, EPOCH_INNER, eval_loader, LR_OUTER, BZ_OUTTER, LR_INNER, BZ_INNER,device=DEVICE, save_path=model_path_reptile)
print(np.mean(loss_reptile))
np.savetxt(f"new_reptile_MSEloss_lro{LR_OUTER}_ei{EPOCH_INNER}.csv", loss_reptile, delimiter=",")
selected_epoch=list(np.linspace(9,50-1,8,dtype=np.int16))

cuda:0
-------------------------MSE_LR_OUTER0.01--EPOCH_INNER3--------------------------
outer_epoch&task0  eval_loss:0.004839874804019928 r2-2.2172040939331055
outer_epoch&task1  eval_loss:0.005143705289810896 r2-2.2441720962524414
outer_epoch&task2  eval_loss:0.005497944541275501 r2-3.0063037872314453
outer_epoch&task3  eval_loss:0.00254303147085011 r2-0.7202330827713013
outer_epoch&task4  eval_loss:0.0019430196844041348 r2-0.36383092403411865
outer_epoch&task5  eval_loss:0.0013554261531680822 r2-0.12543833255767822
outer_epoch&task6  eval_loss:0.0012683873064815998 r2-0.09567487239837646
outer_epoch&task7  eval_loss:0.0016783691244199872 r2-0.05817866325378418
outer_epoch&task8  eval_loss:0.001464937929995358 r2-0.02856576442718506
outer_epoch&task9  eval_loss:0.0015800060937181115 r2-0.051906585693359375
outer_epoch&task10  eval_loss:0.0015937252901494503 r2-0.041413307189941406
outer_epoch&task11  eval_loss:0.0017258659936487675 r2-0.03890204429626465
outer_epoch&task12  eval_loss

In [ ]:
selected_epoch=list(np.linspace(0,epochs_outer-1,5,dtype=np.int16))
plot(loss_reptile, selected_epoch, 'reptile_loss')

In [ ]:

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

model_path_reptile = "./saved_model/reptile_MSE_lro1e-06_ei3.pth"
ns_ls=[16,32,63]
dataset=NomalCo2Dataset_loadALL("./target_task/data_normal")

for ns in ns_ls:
    ns_=63-ns
    print(f"--------------------------NS{ns}lro0.0001_ei3_____________________________")
    dataset_i, _= torch.utils.data.random_split(dataset, [ns, ns_])
    
    model = build_resunetplusplus()
    checkpoint = torch.load(model_path_reptile)
    model.load_state_dict(checkpoint)   
    model = model.to(DEVICE)
    loss_finetune_reptile,mea_finetune_reptile,r2_finetune_reptile,ssim_finetune_reptile=normal_train(model, dataset_i, 0.25, EPOCH, BATCH_SIZE, LR, DEVICE, None)
    
    print(f"-----------------------------------------normalloss_mse_lr{LR}_ns{ns}---------------------------------------------")
    model = build_resunetplusplus()
    model = model.to(DEVICE)
    loss_normal,mea_normal,r2_normal,ssim_normal=normal_train(model, dataset_i, 0.25, EPOCH, BATCH_SIZE, LR, DEVICE,None)

    plot_compare(loss_finetune_reptile, loss_normal, name=f'compare_mse_ns{ns}')
    plot_compare(mea_finetune_reptile, mea_normal, name=f'compare_mea_ns{ns}')
    plot_compare(r2_finetune_reptile, r2_normal, name=f'compare_r2_ns{ns}')
    plot_compare(ssim_finetune_reptile, ssim_normal, name=f'compare_ssim_ns{ns}')

In [ ]:
from loss import all_loss
from para import*
from normal_train import*
time_steps=[1, 3, 6, 9]
import pandas as pd

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)
def plot_sample(sample, time_steps, cmap='YlOrRd', path='log',min=0, max=0.5):
    x = np.arange(1900, 8200, 200)#32 6400
    y = np.arange(1100, 7400, 200)#32 6400
    z = np.arange(-2975, -2820, 10)#16 160
    fig, ax = plt.subplots(1,len(time_steps),figsize=(5*len(time_steps), 5),subplot_kw={'projection': '3d'})
    X,Y,Z = np.meshgrid(x, y, z)
    # min=0
    # max=np.max(sample)
    for i, time_step in enumerate(time_steps):
        grid_t=sample[0,time_step]
        ax[i].set_box_aspect([1, 1, 0.2])
        cur_ax=ax[i].scatter(X[:], Y[:], Z[:],s=100,c=grid_t[:], marker='.', cmap=cmap,vmin=min, vmax=max)
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        ax[i].set_zticks([])
    fig.colorbar(cur_ax,ax=ax,shrink=0.6)
    if path is not None:
        fig.savefig(f"{path}.png",dpi=500)
    fig.show()
    plt.close()


ns_ls=[16,32,63]
model_list=[]
para_log=[]
for ns in ns_ls:
    model_list.append([f"./saved_model/normal_fineds_mse_lr{LR}_ns{ns}.pth",f"./saved_model/finetune_reptilee_MSE_lro0.0001_lri0.001_ns{ns}.pth"])
for i,model_path in enumerate(model_list):
        model = build_resunetplusplus()
        for target_sample in [33,34,36,37,38,51,52,53,58,59,61,62]:
            x=np.load("./target_task/data_normal/all_x.npy")[target_sample]
            y=np.load("./target_task/data_normal/all_y.npy")[target_sample]
            if(i==0):
                plot_sample(y[0:1,:], time_steps, path=f"sample_target_{target_sample}")
            x=torch.tensor(x[0:1,:], dtype=torch.float32).to(DEVICE)
            y=torch.tensor(y[0:1,:], dtype=torch.float32).to(DEVICE)
            y_pred_ls=[]
            y_error_ls=[]
            y_error_max=0
            y_pred_max=0
            for j in range(2):
                checkpoint = torch.load(model_path[j])
                model.load_state_dict(checkpoint)
                model = model.to(DEVICE)
                y_pred = model(x)
                y_error = torch.abs(y_pred - y)
                loss_ms, loss_me, loss_r, loss_s = all_loss(y_pred, y)
                para_log_i={
                        'sample': target_sample,
                        'model_name': model_path[j],
                        'num_sample': ns_ls[i],
                        'ssim': 1-loss_s.data.cpu().numpy(),
                        'mse': loss_ms.data.cpu().numpy(),
                        'mea': loss_me.data.cpu().numpy(),
                        'r^2': loss_r.data.cpu().numpy(),
                    }
                para_log.append(para_log_i)
                print(para_log_i)
                y_error = y_error.cpu().detach().numpy()
                y_pred = y_pred.cpu().detach().numpy()
                y_error_ls.append(y_error)
                y_pred_ls.append(y_pred)
                if np.max(y_error)>y_error_max:
                    y_error_max = np.max(y_error)
                if np.max(y_pred)>y_pred_max:
                    y_pred_max = np.max(y_pred)
            plot_sample(y_pred_ls[0], time_steps, path=f"normal_model_ns{ns_ls[i]}_sample{target_sample}",max=y_pred_max,min=0)
            plot_sample(y_error_ls[0], time_steps, cmap='Reds', path=f"normal_error_ns{ns_ls[i]}_sample{target_sample}",max=y_error_max,min=0)
            plot_sample(y_pred_ls[1], time_steps, path=f"reptile_model_ns{ns_ls[i]}_sample{target_sample}",max=y_pred_max,min=0)
            plot_sample(y_error_ls[1], time_steps, cmap='Reds', path=f"reptile_error_ns{ns_ls[i]}_sample{target_sample}",max=y_error_max,min=0)
           
name=['sample','model_code', 'model_name','ssim','mse','mea','r^2']
test=pd.DataFrame(columns=name,data=para_log)
print(test)
test.to_csv(f"./modellogcsv_{target_sample}.csv",encoding='gbk')

cuda:0
{'sample': 33, 'model_name': './model/normal_fineds_mse_lr0.001_ns16.pth', 'num_sample': 16, 'ssim': 0.8895488977432251, 'mse': array(0.00069924, dtype=float32), 'mea': array(0.00809634, dtype=float32), 'r^2': array(0.45159423, dtype=float32)}
{'sample': 33, 'model_name': './model/finetune_reptilee_MSE_lro0.0001_lri0.001_ns16.pth', 'num_sample': 16, 'ssim': 0.9279865622520447, 'mse': array(0.00023026, dtype=float32), 'mea': array(0.00475256, dtype=float32), 'r^2': array(0.8194066, dtype=float32)}
{'sample': 34, 'model_name': './model/normal_fineds_mse_lr0.001_ns16.pth', 'num_sample': 16, 'ssim': 0.9153656363487244, 'mse': array(0.00079507, dtype=float32), 'mea': array(0.00642757, dtype=float32), 'r^2': array(0.25322002, dtype=float32)}
{'sample': 34, 'model_name': './model/finetune_reptilee_MSE_lro0.0001_lri0.001_ns16.pth', 'num_sample': 16, 'ssim': 0.9286301732063293, 'mse': array(0.00015989, dtype=float32), 'mea': array(0.0042246, dtype=float32), 'r^2': array(0.84981865, dtype